In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import torch
torch.cuda.get_device_name()

In [ ]:
# import wandb

# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="unigram-bert-prothom-alo-epoch6",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 2e-5,
#     "weight_decay" : 0.01,
#     "architecture": "unigram-bert-base",
#     "dataset": "prothom_alo",
#     "epochs": 6,
#     "batch size": 64,
#     }
# )


In [ ]:
from datasets import load_dataset

prothom_alo_dataset = load_dataset("text", data_files="../datasets/Bangla Prothom Alo.txt", split="train")

In [ ]:
prothom_alo_dataset.set_format("pandas")

In [ ]:
prothom_alo_dataset.set_format("pandas")

In [ ]:
prothom_alo_df = prothom_alo_dataset[:]

In [ ]:
prothom_alo_df_lens = prothom_alo_df['text'].str.len()

In [ ]:
max(prothom_alo_df_lens)

In [ ]:
(prothom_alo_df=='').sum()

In [ ]:
# pretraining_df_lens = pretraining_df['text'].str.len()
count = prothom_alo_df['text'].str.split().apply(len).value_counts()

In [ ]:
count.index = count.index.astype(str) + ' words:'
count.sort_index(inplace=True)

In [ ]:
count

In [ ]:
prothom_alo_dataset.reset_format()

In [ ]:
prothom_alo_dataset

In [ ]:
prothom_alo_dataset = prothom_alo_dataset.filter(lambda x: x["text"]!="")

In [ ]:
prothom_alo_dataset

In [ ]:
prothom_alo_dataset[100:200]

In [ ]:
def get_training_corpus():
    for i in range(0, len(prothom_alo_dataset), 1000):
        yield prothom_alo_dataset[i : i+1000]["text"]

In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

In [ ]:
tokenizer = Tokenizer(models.Unigram())

In [ ]:
from tokenizers import Regex

tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace("``", '"'),
        normalizers.Replace("''", '"'),
        normalizers.NFKC(),
#         normalizers.StripAccents(),
        normalizers.Replace(Regex(" {2,}"), " "),
    ]
)

In [ ]:
print(tokenizer.normalizer.normalize_str('রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।'))

In [ ]:
pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)

In [ ]:
# tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

In [ ]:
tokenizer.pre_tokenizer.pre_tokenize_str('রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।')

In [ ]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.UnigramTrainer(
    vocab_size=30522, special_tokens=special_tokens, unk_token="[UNK]"
)

In [ ]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
encoding = tokenizer.encode('রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।')
print(encoding.tokens)

In [ ]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

In [ ]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [ ]:
encoding = tokenizer.encode('রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।')
print(encoding.tokens)

In [ ]:
encoding = tokenizer.encode("এ ছাড়া শিক্ষাপ্রতিষ্ঠানেও চলবে প্রচারণা ।","সহযোগিতা করছে তথ্য ও যোগাযোগপ্রযুক্তি আইসিটি বিভাগ ।")
print(encoding.tokens)
print(encoding.type_ids)

In [ ]:
tokenizer.decoder = decoders.Metaspace()

In [ ]:
tokenizer.decode(encoding.ids)

In [ ]:
tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.save("unigram_tokenizer_prothom_alo.json")

In [ ]:
# tokenizer = Tokenizer.from_file("unigram_tokenizer_prothom_alo.json")

In [ ]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(
    #tokenizer_object=tokenizer,
    tokenizer_file="unigram_tokenizer_prothom_alo.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
    return_special_tokens_mask = True,
    model_max_length = 512,
)

In [ ]:
tokenizer.save_pretrained("unigram_tokenizers_prothom_alo")

In [ ]:
# from transformers import PreTrainedTokenizerFast

# tokenizer = PreTrainedTokenizerFast.from_pretrained("unigram_tokenizers_prothom_alo")

In [ ]:
from transformers import BertConfig, BertForMaskedLM

# Building the config
config = BertConfig()

print(config)

In [ ]:
#test tokenizer
# tokenizer.tokenize("শিক্ষাপ্রতিষ্ঠানেও")
# tokenizer.tokenize("পাচজনকে")
# tokenizer.tokenize("অংশগুলি")

In [ ]:
unigram_bert_config = BertConfig(pad_token_id=tokenizer.pad_token_id)
print(unigram_bert_config)

In [ ]:
unigram_bert_config.save_pretrained(save_directory="configs/unigram_bert_config/")

In [ ]:
unigram_bert_config = BertConfig.from_pretrained("configs/unigram_bert_config/config.json")
# Building the model from the config
# Model is randomly initialized
model = BertForMaskedLM(unigram_bert_config)

In [ ]:
unigram_bert_config

In [ ]:
total_params = 0
for p in model.parameters():
    if len(p.shape) == 2:
        total_params += p.shape[0] * p.shape[1]
        
print(f"Total parameters: {total_params:,}")

In [ ]:
# text = 'পরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।'
text = 'পরে সেখানে সংক্ষিপ্ত সমাবেশ [MASK] হয় ।'

In [ ]:
tokenizer.tokenize(text)

In [ ]:
import torch 

inputs = tokenizer(text, return_tensors="pt")
# inputs.to("cuda")

token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["text"], padding="max_length", max_length=80, truncation=True)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = prothom_alo_dataset.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets

In [ ]:
tokenized_datasets.remove_columns("token_type_ids")

In [ ]:
# temp = tokenized_datasets.filter(lambda x:x if 1 in x["input_ids"] else None)

In [ ]:
tokenized_datasets[0]

In [ ]:
def group_texts(examples):
    # Create a new labels column
    examples["labels"] = examples["input_ids"].copy()
    return examples

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

In [ ]:
tokenizer.decode(lm_datasets[1]["input_ids"])

In [ ]:
tokenizer.decode(lm_datasets[1]["labels"])

In [ ]:
tokenizer.pad_token_id

In [ ]:
import collections
import numpy as np
np.random
from transformers import default_data_collator

wwm_probability = 0.15


def bangla_data_collator(features):
    for feature in features:
#         word_ids = feature.pop("word_ids")

#         # Create a map between words and corresponding token indices
#         mapping = collections.defaultdict(list)
#         current_word_index = -1
#         current_word = None
#         for idx, word_id in enumerate(word_ids):
#             if word_id is not None:
#                 if word_id != current_word:
#                     current_word = word_id
#                     current_word_index += 1
#                 mapping[current_word_index].append(idx)

        # Randomly mask words
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        mask = np.random.binomial(1, wwm_probability, (len(input_ids),))
        special_tokens =  [tokenizer.unk_token_id, tokenizer.pad_token_id, tokenizer.cls_token_id, \
                           tokenizer.sep_token_id, tokenizer.mask_token_id]
        
        new_labels = [-100] * len(labels)
        for idx in np.where(mask)[0]:
#             word_id = word_id.item()
#             print(word_id)
#             for idx in mapping[word_id]:
#             if word_ids[idx] is not None:
            if input_ids[idx] not in special_tokens:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
            feature["labels"] = new_labels
        
    return default_data_collator(features)

In [ ]:
from transformers import DataCollatorForLanguageModeling

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
lm_datasets = lm_datasets.remove_columns(["word_ids"])
data_collator = bangla_data_collator

In [ ]:
samples = [lm_datasets[i] for i in range(3)]
# for sample in samples:
#     _ = sample.pop("word_ids")

for chunk in bangla_data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

In [ ]:
prothom_alo_dataset[:3]

In [ ]:
samples = [lm_datasets[i] for i in range(1)]

chunk = data_collator(samples)
print(chunk["input_ids"])
print(chunk["labels"])

In [ ]:
# train_size = 10_000
# test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets.train_test_split(
    train_size=0.8, seed=42
)
downsampled_dataset

In [ ]:
for idx, sample in enumerate(downsampled_dataset["train"]["input_ids"][:3]):
    print(f"'>>> Article {idx} length: {len(sample)}'")

In [ ]:
tokenizer.decode(downsampled_dataset["test"]["input_ids"][0])

In [ ]:
#  disable weights and biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# from __future__ import division

In [ ]:
from transformers import TrainingArguments

batch_size = 64
eval_batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size


training_args = TrainingArguments(
    num_train_epochs = 3,
    report_to = None,
    output_dir="models/unigram/bert-base-pretrained-prothom-alo",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    #push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
    load_best_model_at_end=True,
    save_strategy = "epoch",
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.save_model()

In [ ]:
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

In [ ]:
model = BertForMaskedLM.from_pretrained("models/unigram/bert-base-pretrained-prothom-alo")

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("models/unigram/bert-base-pretrained-prothom-alo")

In [ ]:
import torch
torch.cuda.empty_cache()